# <center>Graph Search</center>

## Graph Search And Connectivity

In [1]:
# Imports 
import os 

path_file = '../Data/SCC.txt'
filepath = os.path.join(os.getcwd(), path_file)

In [2]:
def read_file(name):
    
    num_nodes = 875715
    file = open(name,"r")
    data = file.readlines()
    
    G = [[] for i in range(num_nodes)]
    G_rev = [[] for i in range(num_nodes)]
    
    
    for line in data:
        items = line.split()
        G[int(items[0])] += [int(items[1])]
        G_rev[int(items[1])] += [int(items[0])]
        
    return G, G_rev

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/Users/stephanepeillet/Downloads/Download.CSV')
result = df[df['Type'] == 'Virement par carte bancaire standard'].reset_index()

In [36]:
df.shape

(184, 41)

Index(['index', 'Date', 'Heure', 'Fuseau horaire', 'Nom', 'Type', 'État',
       'Devise', 'Avant commission', 'Commission', 'Net', 'De l'adresse email',
       'À l'adresse email', 'Numéro de transaction', 'Adresse de livraison',
       'État de l'adresse', 'Titre de l'objet', 'Numéro de l'objet',
       'Montant des frais d'expédition et de traitement',
       'Montant de l'assurance', 'TVA', 'Nom de l'option 1',
       'Valeur de l'option 1', 'Nom de l'option 2', 'Valeur de l'option 2',
       'Numéro de la transaction de référence', 'Numéro de facture',
       'Numéro de client', 'Quantité', 'Numéro de reçu', 'Solde', 'Adresse',
       'Adresse (suite)/District/Quartier', 'Ville',
       'État/Province/Région/Comté/Territoire/Préfecture/République',
       'Code postal', 'Pays', 'Numéro de téléphone du contact', 'Objet',
       'Remarque', 'Indicatif pays', 'Impact sur le solde'],
      dtype='object')

In [23]:
new = result.drop(columns=['index', 'Fuseau horaire', 'Nom', 'Type', 'État',
       'Devise', 'Avant commission', 'Commission', 'Net', "De l'adresse email",
       "À l'adresse email", 'Numéro de transaction', 'Adresse de livraison',
       "État de l'adresse", "Titre de l'objet", "Numéro de l\'objet",
       "Montant des frais d'expédition et de traitement",
       "Montant de l'assurance", "TVA", "Valeur de l\'option 2",
       'Numéro de la transaction de référence', 'Numéro de facture',
       'Numéro de client', 'Quantité', 'Numéro de reçu', 'Solde', 'Adresse',
       'Adresse (suite)/District/Quartier', 'Ville',
       'État/Province/Région/Comté/Territoire/Préfecture/République',
       'Code postal', 'Pays', 'Numéro de téléphone du contact',
       'Remarque', 'Indicatif pays', 'Impact sur le solde'])

In [35]:
new

,Date,Heure,Nom de l'option 1,Valeur de l'option 1,Nom de l'option 2,Objet
0,18/07/2019,18:27:00,NaN,NaN,NaN,Verbling Tutoring
1,21/07/2019,23:30:26,NaN,NaN,NaN,NaN
2,29/07/2019,13:27:44,NaN,NaN,NaN,NaN
3,01/08/2019,22:04:28,NaN,NaN,NaN,Masmusculo black card
4,21/08/2019,16:27:54,NaN,NaN,NaN,6222332_1051
...,...,...,...,...,...,...
64,30/10/2020,01:11:02,NaN,NaN,NaN,Préparation pour Mug Cake Protéiné - 500g - Ch...
65,01/11/2020,21:37:15,NaN,NaN,NaN,Masque protecteur extérieur unisexe avec 2 fil...
66,05/11/2020,11:13:12,NaN,NaN,NaN,NaN
67,15/11/2020,21:01:02,NaN,NaN,NaN,Collagène Peptide Peptan® 1 (tendons & articul...


In [34]:
result 

KeyError: '["Nom\\\\ de\\\\ l\'option\\\\ 1"] not found in axis'